Implémentation d'une architecture de classification multimodale à partir de classifieurs unimodaux.

Diverses stratégies de fusion pour la jointure des informations : précoce et tradive !

In [ ]:
from google.colab import drive
drive.mount('drive', force_remount=True)

Mounted at drive


In [ ]:
# UPDATE user_id WITH YOUR OWN PATH
users = ["William", "Alice", "Vincent"]
user_id = 1

if user_id == 0:
  path = '/content/drive/MyDrive/CENTRALE MARSEILLE/3A/SAM/Projet/data/msdi'  # Should lead to msdi folder
  text_path = '/content/drive/MyDrive/CENTRALE MARSEILLE/3A/SAM/Projet/data/texte'  #Should lead to texte folder
if user_id == 1:
  path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi'
  text_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/texte'
if user_id == 2:
  path = '/content/drive/MyDrive/Projet/data/msdi'
  text_path = '/content/drive/MyDrive/Projet/data/texte'  

msdi_path = path  # Don't ask : flemme de modifier les variables dessous

#Import des données 

####Imports en tout genre

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from tqdm import tqdm

####Étiquettes

In [ ]:
# Fonction pour charger la liste des étiquettes (les genres)
def get_label_list(msdi_path):
    df = pd.read_csv(Path(msdi_path) / 'labels.csv', header=None)
    return list(df.iloc[:, 0])

#msdi_path = '/content/drive/MyDrive/CENTRALE MARSEILLE/3A/SAM/Projet/data/msdi' # à modifier si chemin différent

labels = get_label_list(msdi_path)

# Encode les genres en entiers
labelEncoder = LabelEncoder()
labels = labelEncoder.fit_transform(labels)
labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

####Audio

*load_deep_audio_features* est la fonction qui, à partir d'une entrée (ligne dans le mapping) et du chemin vers le mapping (msdi_path), charge une représentation audio du morceau ! </br>
- representation_morceau_i = load_deep_audio_features(entry, msdi_path)

In [ ]:
#%cd /content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/out_audio # Modifier avec son chemin
#from representations import get_x_y_train_test, load_deep_audio_features # Fonction contenue dans un module python

In [ ]:
#msdi_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/' # Modifier avec son chemin
#X_train, X_test, y_train, y_test = get_x_y_train_test(msdi_path) # 1. ≃ 20 secondes ; 2. ≃ 1 minutes 30 ; 3. ≃ 20 secondes

In [ ]:
#X_train[0].shape # Taille d'un descripteur = 2048

####Images

In [ ]:
# path = '/content/drive/MyDrive/CENTRALE MARSEILLE/3A/SAM/Projet/data/msdi'
# df = pd.read_csv(path+'/msdi_mapping.csv')

# df_train = df[df['set']=='train']
# df_test = df[df['set']=='test']

# nb_train_img = len(df_train)
# nb_test_img = len(df_test)

# print('Nombre images de train : ', nb_train_img)
# print('Nombre images de test : ', nb_test_img)

####Textes

In [ ]:
# #text_path = '/content/drive/MyDrive/Projet/data/texte'

# # Importe les données sous la forme :
# # Ligne de la chanson, track id, genre, vecteur "bag of word"
# df_text = pd.read_csv(text_path + '/lyrics_genres_structured.csv')

# # On suppprime les colonnes : n° chanson et son id
# df_text.columns
# df_genres_and_bags = df_text.drop(columns=["Unnamed: 0", "msd_track_id"])

# # DataFrame contenant que les bags of words
# df_bags = df_genres_and_bags.drop('genre', axis = 1)

# # DataFrame contenant que les labels
# df_genres = df_genres_and_bags['genre']

# # X - Bag of words
# X = np.array(df_bags)

# # Y - Label (genres encodés)
# #Label encoder
# label_encoder = preprocessing.LabelEncoder()
# label_encoded = label_encoder.fit_transform(df_genres)

# #One hot encoder
# y = tf.keras.utils.to_categorical(label_encoded, num_classes=15)

# # Séparation des données d'entraînements et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Compatibilité des dataframes 

In [ ]:
df_msdi = pd.read_csv(path+'/msdi_mapping.csv')
df_text = pd.read_csv(text_path + '/lyrics_genres_structured.csv')

In [ ]:
# # On compte le nombre de chansons en communs 
# cmpt = 0
# for track1 in df_msdi['msd_track_id']:
#     for track2 in df_text['msd_track_id']:
#         if track1 == track2:
#             cmpt += 1
# print(cmpt)

#### Filtrage des dataframes

Création df_train et df_test

In [ ]:
#df_train = df_msdi.copy()
#df_test = df_msdi.copy()

#n = len(df_msdi)  #Display purpose only

# for index, track1 in enumerate(df_msdi['msd_track_id']):
#     if index%5000 == 0: print(index,'/', n)
#     found = False
#     for track2 in df_text['msd_track_id']:
#         if track1 == track2:  #sample inside both datasets
#             found = True
#             type_set = df_msdi[df_msdi['msd_track_id'] == track1].drop(columns=["Unnamed: 0"])['set'].values[0]
#             #type_set = row['set']#[1]
#             #print(type_set.values)

#             if type_set == "train":
#               df_test = df_test.drop(index)
#             if type_set == "test":
#               df_train = df_train.drop(index)

#     if not found:
#       df_train = df_train.drop(index)
#       df_test = df_test.drop(index)

In [ ]:
# Sauvegarde les dataframe de test et train
#train_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/out_multimodal/train.csv'
#test_path = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/out_multimodal/test.csv'
#df_train.to_csv(train_path, index=False)
#df_test.to_csv(test_path, index=False)
df_train = pd.read_csv(path[:-5]+'/out_multimodal/train.csv')
df_test = pd.read_csv(path[:-5]+'/out_multimodal/test.csv')

In [ ]:
nb_samples_test = len(df_test)
nb_samples_train = len(df_train)

print("Number samples train :", nb_samples_train)
print("Number samples test :", nb_samples_test)

Number samples train : 12638
Number samples test : 4410


#Import des architectures unimodales

In [ ]:
import tensorflow as tf
import keras
from keras.models import Model

####Modèle audio

In [ ]:
new_path = path + '/out_audio'
%cd $new_path
from model import load_model # Fonction contenue dans un module python

/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/msdi/out_audio


In [ ]:
model_path = new_path + '/audio_model' # Modifier avec son chemin
audio_model = load_model(model_path) # Modèle unimodal pour les fichiers audios

####Modèle image

In [ ]:
from tensorflow.keras.models import load_model
model_image = load_model(path + '/out_img/cnn_model_images.h5')
imput_shape_img = (200, 200, 3)
model_image.summary()

Model: "CNNClassifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 200, 200, 128)     3584      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 67, 67, 128)      0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 67, 67, 128)      512       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 67, 67, 64)        73792     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                     

In [ ]:
# Modèle encodeur (génère un représentation à partir d'une image)
img_encoder = Model(inputs=model_image.input, outputs=model_image.get_layer('flatten_2').output)

# Modèle classifieur (génère une distribution de probabilité parmis les classes à prédire, à partir d'une représentation d'image)
img_clf = Model(inputs=model_image.get_layer('dense_6').input, outputs=model_image.output)

# Scaler pour les images
from joblib import load
scaler_img = load(path+'/out_img/'+f'deepScaler-Images{imput_shape_img[0]}.z')

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


####Modèle texte

In [ ]:
# Import du model : text_classifier
text_classifier = keras.models.load_model(text_path + '/out_text/text_classifier.h5')

In [ ]:
text_classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                320064    
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 15)                495       
                                                                 
Total params: 322,639
Trainable params: 322,639
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(text_classifier.input_shape)

(None, 5000)


#Fonctions

In [ ]:
from sklearn.utils import shuffle
from keras.utils import np_utils
from tensorflow.keras.preprocessing import image

Méthode de chargement

Va accéder aux données pour les importer sur le notebook

In [ ]:
def loadImageANDLabel(row):
    '''
    Charge une image à partir de son nom
    Reshape l'image pour fit l'entrée du modèle
    '''
    img_name = row['img']
    img_path = path +'/'+img_name
    img = image.load_img(img_path, target_size=(imput_shape_img[0], imput_shape_img[1]))

    label_name = row['genre']
    label_idx = int(labelEncoder.transform([label_name]))
    # hot_enc_label = np_utils.to_categorical([label_idx], num_classes=15)

    #TODO return img FEATURES
    return img, label_idx


def loadAudioFeatures(row):
    '''
    load_deep_audio_features
    '''
    subset_file = 'X_{}_audio_MSD-I.npy'.format(row['set'])
    x = np.load(Path(path) / 'deep_features' / subset_file, mmap_mode='r')
    idx = row['deep_features']
    return x[idx, :]

def loadTextFeatures(row):
    df_txt_row = df_text[df_text['msd_track_id'] == row['msd_track_id']]
    feat_df = df_txt_row.drop(columns=["Unnamed: 0", "msd_track_id", "genre"])
    feat = np.array(feat_df)
    return feat

Data Loader

Charge en mémoire des batchs (dans des variables)

In [ ]:
def load_data(df_batch, batch_size): #set_indices
    """
    Import *batch_size* representations (for each data type image/sound/text) from files and stock them into variables
    """

    imgs = np.empty((batch_size, imput_shape_img[0], imput_shape_img[1], imput_shape_img[2]))
    batch_txt = np.empty((batch_size, 5000))
    batch_audio = np.empty((batch_size, 2048))
    one_hot_labels = np.empty((batch_size,15))
    batch_labels=np.empty(batch_size)

    
    for j in range(len(df_batch.index)) :
      row = df_batch.iloc[j,:]

      #IMAGE Chargement des labels et images (pas encore features)
      img, label_idx = loadImageANDLabel(row)
      imgs[j] = img
      batch_labels[j] = int(label_idx)

      #SOUND Chargement des labels et features
      feat = loadAudioFeatures(row)
      batch_audio[j] = feat

      #TEXT Chargement des labels et features
      feat = loadTextFeatures(row)
      batch_txt[j] = feat

    # Hot encoding labels
    for i,class_id in enumerate(batch_labels):
      one_hot_labels[i] = np_utils.to_categorical(batch_labels[i], num_classes=15)
      #batch_labels = np_utils.to_categorical(batch_labels, num_classes=15)
    
    # Création images features : Traitement des images plus rapide par batch
    # 1 - Normalisation des images
    imgs = scaler_img.transform(imgs.reshape((batch_size, imput_shape_img[0] * imput_shape_img[1] * imput_shape_img[2]))) \
                  .reshape((batch_size, imput_shape_img[0], imput_shape_img[1], imput_shape_img[2])) 
    # 2 - Récupération des features images (sortie encoder)
    batch_img = img_encoder.predict(imgs)

    return batch_img, batch_audio, batch_txt, one_hot_labels # was batch_labels before

Générateur

In [ ]:
def get_batch(mode, batch_size, shuffleDataset=True):
    if mode=='train' : 
      nbdata = len(df_train.index)
      df = df_train.copy()
    elif mode=='test': 
      nbdata = len(df_test.index)
      df = df_test.copy()

    if shuffleDataset : 
      df = shuffle(df).reset_index(drop=True)
    else : df.reset_index(drop=True)

    i = 0
    while True:
      i += batch_size
      if i+batch_size > nbdata: i = 0
      
      df_batch=df.iloc[i*batch_size : (i+1)*batch_size]
      yield load_data(df_batch=df_batch, batch_size=batch_size)

In [ ]:
#Test fonctions
data_generator_test = get_batch('test', 2)
x_img, x_audio, x_txt, y = next(data_generator_test)
# # print(y[0])
# plt.figure(0)
# plt.imshow(x_img[0])

print('img shape :', x_img.shape)
print('audio shape :', x_audio.shape)
print('texte shape :', x_txt.shape)
print('label shape :', y.shape)

img shape : (2, 576)
audio shape : (2, 2048)
texte shape : (2, 5000)
label shape : (2, 15)


#Stratégies de fusion

##Fusion précoce

Définition fonctions

In [ ]:
#import pickle

# Fonction qui permet de sauvegarder le modèle
#def save_model(model,model_path):
  #pickle.dump(model, open(model_path, "wb"))

def train_early(model, mode, batch_size, iterations=1):
  """
  Fonction qui entraîne un modèle sur des données multimodales agrégée d'une certaine manière
  """
  data_generator_train = get_batch('train', batch_size)
  nombre_batchs = int(nb_samples_train/batch_size)
  history_loss = []
  for iteration in range(iterations):
    for batch_id in tqdm(range(nombre_batchs)):
      # Génère les représentations pour chaque modalité du batch
      x_image, x_audio, x_texte, label = next(data_generator_train)
      # Concatène les représentations
      if mode == "concat":
        input = np.concatenate((x_image, x_audio), axis = 1)
        input = np.concatenate((input, x_texte), axis = 1)
      elif mode == "add":
          x_image_mod = np.resize(x_image, (50,5000)) #trim_zeros pourrait marcher aussi
          x_audio_mod = np.resize(x_audio, (50,5000))
          input = x_image_mod + x_audio_mod + x_texte
      elif mode == "multiply":
          x_image_mod = np.resize(x_image, (50,5000))
          x_audio_mod = np.resize(x_audio, (50,5000))
          input = x_image_mod * x_audio_mod * x_texte
      # Entraînement du modèle sur le batch
      loss = model.train_on_batch(input, label)
      print(f"  Loss for the batch {batch_id} : ", loss)  #, np.isnan(x_image), np.isnan(x_audio), np.isnan(x_texte), np.isnan(label)
      history_loss.append(loss)
    iteration_loss = np.mean(history_loss)
    print("Iteration loss : ", iteration_loss)
  return model

def getClassifier(input_shape):
  nombre_genres = 15
  from tensorflow.keras.models import Model, Sequential
  from tensorflow.keras.layers import Dense
  from tensorflow.keras import optimizers

  model = Sequential(name="Classifier")
  model.add(Dense(256, activation='relu', input_shape = input_shape))
  #model.add(Dense(128, activation='relu'))
  model.add(Dense(nombre_genres, activation='softmax'))
  model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

Entrainement

In [ ]:
#from sklearn.neural_network import MLPClassifier

#############
# IMPORTANT # Adapter le path2 à votre architecture (uniquement pour load ou sauvegarde du modèle)
#############
path2 = '/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data' # ALICE
# "/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data" WILLIAM


# Phase d'entraînement
batch_size = 3
representation_size_cont = 576 + 2048 + 5000
representation_size_add = 5000
representation_size_multi = 5000
input_shape = (representation_size_cont,)
#input_shape = (batch_size, representation_size)
agg_mode = "concat"
"""
mode
= "concat'
= "add" #TODO voir pour mettre features de meme taille
= "multiply"

ATTENTION : il faut que le modèle utilisé soit compatible avec le mode demandé:
un modèle utilisant le mode 'concat' aura une entrée plus grande que ceux qui utiliseront un mode 'add' ou 'multiply'
-> Cela est pris en charge par le try /except si dessous
"""
#Chargement ou création (si modèle non existant) du modèle associé au mode d'aggrégation voulu
#try:
  #model = load_model(path2 + f'/out_multimodal/model_early_{agg_mode}')
  #print(f"Récupération du modèle early existant avec mode {agg_mode}")
#except:
model = getClassifier(input_shape)
  #MLPClassifier(random_state=1, max_iter=300, activation='softmax')
print(f"Création du modèle early avec mode {agg_mode}")

# Entraîne le modèle
clf = train_early(model, agg_mode, batch_size, iterations=1)

Création du modèle early avec mode concat


  0%|          | 1/4212 [00:01<1:33:03,  1.33s/it]

  Loss for the batch 0 :  [34.57498550415039, 0.0]


  0%|          | 3/4212 [00:02<54:59,  1.28it/s]  

  Loss for the batch 1 :  [6.187838077545166, 0.3333333432674408]
  Loss for the batch 2 :  [32.509037017822266, 0.0]


...

  Loss for the batch 4206 :  [0.003834998933598399, 1.0]
  Loss for the batch 4207 :  [0.003922760486602783, 1.0]
  Loss for the batch 4208 :  [0.0038295413833111525, 1.0]


100%|█████████▉| 4211/4212 [15:50<00:00, 12.08it/s]

  Loss for the batch 4209 :  [0.003917304333299398, 1.0]
  Loss for the batch 4210 :  [0.0038239257410168648, 1.0]


100%|██████████| 4212/4212 [15:50<00:00,  4.43it/s]

  Loss for the batch 4211 :  [3.792731523513794, 0.6666666865348816]
Iteration loss :  1.617320895050345


In [ ]:
# import pickle

# # Fonction qui permet de sauvegarder le modèle
# def save_model(model,model_path):
#     pickle.dump(model, open(model_path, "wb"))
agg_mode = "concat"
model_path = path[:-5] + f'/out_multimodal/model_early_{agg_mode}'
# f'/content/drive/MyDrive/Colab Notebooks/Temps2/SAM/Projet/data/out_multimodal/model_early_{agg_mode}'
# save_model(clf, model_path)
# clf.save(model_path)

In [ ]:
loaded_model = tf.keras.models.load_model(model_path)

Evaluation

In [ ]:
def accuracy(y_true, y_pred):
  class_id_pred = np.argmax(y_pred, axis=1)
  class_id_true = np.argmax(y_true, axis=1)
  count = 0
  for i in range(len(class_id_true)):
    if class_id_pred[i] == class_id_true[i]:
      count+=1
  return count / len(class_id_true)



# Donne la precision du modele
# accuracy = accuracy(y_true, y_pred)
# print("La précision du modèle est de : ", 100*accuracy, " %")


def eval_early(model, mode, batch_size):
    """
    Fonction qui calcule l'accuracy d'un modèle sur des données multimodales agrégée d'une certaine manière
    """
    data_generator_test = get_batch('test', batch_size)
    list_acc = []

    # Génère les représentations pour chaque modalité du batch
    x_image, x_audio, x_texte, label = next(data_generator_test)
    # Concatène les représentations
    if mode == "concat":
      input = np.concatenate((x_image, x_audio), axis = 1)
      input = np.concatenate((input, x_texte), axis = 1)
    elif mode == "add":
        x_image_mod = np.resize(x_image, (50,5000)) #trim_zeros pourrait marcher aussi
        x_audio_mod = np.resize(x_audio, (50,5000))
        input = x_image_mod + x_audio_mod + x_texte
    elif mode == "multiply":
        x_image_mod = np.resize(x_image, (50,5000))
        x_audio_mod = np.resize(x_audio, (50,5000))
        input = x_image_mod * x_audio_mod * x_texte
    # Prédiction du modèle sur le batch
    print(input)
    print(label)
    y_pred = model.predict(input)
    #Calcul accuracy pour le batch
    acc = accuracy(y_true=label, y_pred=y_pred)
  
    return  acc

In [ ]:
acc = eval_early(model=loaded_model, mode="concat", batch_size=len(df_test))
print(acc)

NameError: ignored

In [ ]:
#TODO
def eval():
  return

In [ ]:
# batch_size = 50
# nb_batches = int(len(df_test)/batch_size)
# data_generator_test = get_batch('test', batch_size)
# nombre_genres = 15
# mode = "add"

# labels = []
# y_batch = []
# for batch_id in tqdm(range(nb_batches)):

#   #Génération du batch de chaque modalité
#   x_image, x_audio, x_texte, label = next(data_generator_test)
#   # Concatène les représentations
#   if mode == "concat":
#     input = np.concatenate((x_image, x_audio), axis = 1)
#     input = np.concatenate((input, x_texte), axis = 1)
#   elif mode == "add":
#     x_image_mod = np.resize(x_image, (50,5000)) #trim_zeros pourrait marcher aussi
#     x_audio_mod = np.resize(x_audio, (50,5000))
#     input = x_image_mod + x_audio_mod + x_texte
#   elif mode == "multiply":
#     x_image_mod = np.resize(x_image, (50,5000)) #trim_zeros ne peut pas marcher (multiplication par 0 = perte de l'information)
#     x_audio_mod = np.resize(x_audio, (50,5000))
#     input = x_image * x_audio * x_texte
#   # Calcule les prédictions du classifieur pour la représentation concaténée
#   #try:
#   fusion_pred = loaded_model.predict(input)
#   print(f"Pour le batch numéro {batch_id} on a la prédiction : {fusion_pred}")
#   y_batch.append(fusion_pred)
#   labels.append(label)
  #except:
    #pass

  0%|          | 0/88 [00:12<?, ?it/s]


ValueError: ignored

In [ ]:
dimension = np.array(labels).shape[0] * batch_size
y_true = np.array(labels).reshape(dimension, nombre_genres)
y_pred = np.array(y_batch).reshape(dimension, nombre_genres)

In [ ]:
# for i,row in enumerate(y_pred):
#   if np.isnan(row).any():
#     print(f"The row {i} is a NaN")
#     print(row)

In [ ]:
y_pred = y_pred[60:119]
y_true = y_true[60:119]

In [ ]:
# from sklearn.metrics import accuracy_score

# acc = accuracy_score(y_true, y_pred) * 100
# acc

In [ ]:
def accuracy(y_true, y_pred):
  class_id_pred = np.argmax(y_pred, axis=1)
  class_id_true = np.argmax(y_true, axis=1)
  count = 0
  for i in range(len(class_id_true)):
    if class_id_pred[i] == class_id_true[i]:
      count+=1
  return count / len(class_id_true)

from sklearn.metrics import roc_auc_score

# Donne la precision du modele
accuracy = accuracy(y_true, y_pred)
print("La précision du modèle est de : ", 100*accuracy, " %")

# Donne la précision du modele
#my_roc_score = roc_auc_score(label, fusion_pred)
#print("Le ROC score du modèle après fusion tardive est de : ", 100*my_roc_score, " %")

##Fusion tardive

Méthode d'ensemble de type *bagging* où la décision finale est déterminée par pondération des décisions des classifieurs unimodaux.

In [ ]:
from sklearn.metrics import roc_auc_score

# Fonction qui renvoie le softmax d'un vecteur
def softmax(vecteur):
  exponentielle = np.exp(vecteur)
  poids = exponentielle / np.sum(exponentielle)
  return poids

# Fonction qui calcule la précision d'un modèle
def my_accuracy(y_true, y_pred):
  class_id_pred = np.argmax(y_pred, axis=1)
  class_id_true = np.argmax(y_true, axis=1)
  count = 0
  for i in range(len(class_id_true)):
    if class_id_pred[i] == class_id_true[i]:
      count+=1
  return count / len(class_id_true)

# Fonction qui renvoie le vecteur des scores de certitude pour chaque classifieur
def getClassifierConfidence(predictions):
  """Returns model confidence in it's batch prediction"""
  confidences = []
  for sample_pred in predictions:
    second_max, first_max = np.sort(sample_pred)[-2:]
    confidence = first_max - second_max
    confidences.append(confidence)
  # Possibility to get min, mean or max confidence
  return np.mean(confidences)

def test_late(mode):
  dimension = len(df_test)

  data_generator_test = get_batch('test', dimension)

  print("...Generating representations...")
  print()
  x_image, x_audio, x_texte, label = next(data_generator_test)
  print("DONE generating representations !")
  print()

  image_pred = img_clf.predict(x_image)
  if np.isnan(image_pred).any() or np.isinf(image_pred).any():
    print("NaN or Infinite value in the images predictions")
    print()
  audio_pred = audio_model.predict_proba(x_audio)
  if np.isnan(audio_pred).any() or np.isinf(audio_pred).any():
    print("NaN or Infinite value in the audio predictions")
    print()
  texte_pred = text_classifier.predict(x_texte)
  if np.isnan(texte_pred).any() or np.isinf(texte_pred).any():
    print("NaN or Infinite value in the texts predictions")
    print()
  
  if mode == "roc":
    score_image = 0.5
    score_audio = 0.83
    score_texte = 0.78

  elif mode == "confidence":
    score_image = getClassifierConfidence(image_pred)
    score_audio = getClassifierConfidence(audio_pred)
    score_texte = getClassifierConfidence(texte_pred)

  vecteur_score = np.array([score_image, score_audio, score_texte])

  # Calcule le poids associé à chaque classifieur
  poids = softmax(vecteur_score)
  poids_image = poids[0]
  poids_audio = poids[1]
  poids_texte = poids[2]
  
  nombre_genres = 15

  fusion_pred = np.empty((dimension, nombre_genres))
  for i in range(dimension):
      fusion_pred[i] = softmax(poids_audio * audio_pred[i] + poids_texte * texte_pred[i])

  # Donne la precision du modele
  acc = my_accuracy(label, fusion_pred)
  print("La précision du modèle après fusion tardive est de : ", 100*acc, " %")

  my_roc_score = roc_auc_score(label, fusion_pred)
  print("Le ROC score du modèle après fusion tardive est de : ", 100*my_roc_score, " %")

In [ ]:
mode = "confidence" # roc # confidence

# Évalue le modèle de fusion tardive
test_late(mode)

...Generating representations...



**Pondération ROC**
- La précision du modèle après fusion tardive est de :  86.16780045351474  %
- Le ROC score du modèle après fusion tardive est de :  96.86491579889372  %


**Pondération "confidence"**
- La précision du modèle après fusion tardive est de :  87.14285714285714  %
- Le ROC score du modèle après fusion tardive est de :  97.11456440910254  %